In [1]:
from neo4j import GraphDatabase

In [16]:
uri = 'neo4j://localhost:7687'
user = 'neo4j'
password = '1111'

In [17]:
driver = GraphDatabase.driver(uri, auth=(user, password))

In [18]:
def run_query(func):    
    with driver.session() as session:         
        session.read_transaction(func)   

In [35]:
def get_cypher_method(tx):
    lst = []
    query = """
               MATCH (a{명칭:'암호화방식'}) - [r:종류] -> (b)
               RETURN b.명칭 AS 암호화방식종류
            """
    result = tx.run(query)    
    
    for record in result:
        lst.append(record['암호화방식종류']) 

    print('암호화방식의 종류에는', ', '.join(lst), '등이 있습니다.')
    
run_query(get_cypher_method)      

암호화방식의 종류에는 비대칭키암호, 해시함수, 대칭키암호 등이 있습니다.


---

In [73]:
def get_symetric_key(tx):
    lst = []
    query = """
        MATCH (a{명칭:'대칭키암호'}) - [r:종류] -> (b)
        RETURN b.명칭 AS 명칭, b.키길이 AS 키길이
        """
    result = tx.run(query)
    for record in result:
        lst.append('{}(키길이:{})'.format(record['명칭'], record['키길이']))
    print('대칭키 암호 알고리즘은', ''.join(lst),'등이 있습니다.')
run_query(get_symetric_key)

대칭키 암호 알고리즘은 AES(키길이:128, 192, 256)ARIA(키길이:128, 192, 256)SEED(키길이:128, 256) 등이 있습니다.


---

In [65]:
def get_primary_key(tx):
    query = """
        MATCH (a) - [r:유사어] -> (b{명칭 : '비밀키암호'})
        RETURN a.설명 AS 설명
        """
    result = tx.run(query)
    for record in result:
        print('비밀키 암호 알고리증은', record['설명'], '입니다.')
    
    lst = []
    query ="""
                MATCH (a) - [rl: 유사어] -> (b{명칭:'비밀키암호'})
                MATCH (a) - [r2: 종류] -> (c)
                RETURN a.명칭 AS 명칭
            """
    result = tx.run(query)
    for record in result:
        lst.append (record['명칭'])
    
    print('비일키 알고리즘의 종류에는', ', '.join(lst), '등이 있습니다.')

run_query(get_private_key)

비밀키 암호 알고리즘은 암호화에 사용되는 키(암호화 키)와 복호화에 사용되는 키(복호화 키)가 서로 동일한 암호화 알고리즘 입니다.
비밀키 알고리즘의 종류에는 AES, ARIA, SEED 등이 있습니다.


---

In [56]:
def run_exec_query(func):    
    with driver.session() as session:         
        session.write_transaction(func)  

In [54]:
def get_Hash(tx):
    lst = []
    query = """
               MATCH (a{명칭:'해시함수'}) - [r:종류] -> (b)
               RETURN b.명칭 AS 명칭
               ORDER BY 명칭
            """
    result = tx.run(query)    
    for record in result:
        lst.append(record['명칭']) 
    print('해시함수 종류에는', ', '.join(lst), '등이 있습니다.')
    ㅇ
run_query(get_Hash) 

해시함수 종류에는 SHA 등이 있습니다.


In [57]:
def set_Hash(tx):
    
    get_Hash(tx)
    
    query = """
               CREATE (a:암호화 { 명칭 : 'SHA3', 출력길이 : '224, 256, 384, 512', \
               설명 : 'SHA-1과 SHA-2를 대체하기 위해 미국 국립표준기술연구소(NIST)가 2015년 8월 5일에 발표한 스펀지 구조의 암호화 해시함수' });
            """
    result = tx.run(query)    
    
    query = """
               MATCH (a{명칭:'해시함수'}) 
               MATCH (b{명칭:'SHA3'}) 
               CREATE (a) - [r:종류] -> (b);
            """
    result = tx.run(query)    

    get_Hash(tx)
        
run_exec_query(set_Hash)  

해시함수 종류에는 SHA 등이 있습니다.
해시함수 종류에는 SHA, SHA3 등이 있습니다.


---